In [159]:
import xml.dom.minidom
import urllib

def getText(nodelist):
    rc = []
    for node in nodelist:
        if node.nodeType == node.TEXT_NODE:
            rc.append(node.data)
    return ''.join(rc)


search = xml.dom.minidom.parse(open(r"C:\Users\shaik\PycharmProjects\indeed\search_criteria.xml"))

search_string = getText(search.getElementsByTagName("query")[0].childNodes)
search_location = getText(search.getElementsByTagName("location")[0].childNodes)
publisher_key = getText(search.getElementsByTagName("key")[0].childNodes)

out_final = {}

for i in range(0,39) :
    req = "http://api.indeed.com/ads/apisearch?publisher="+publisher_key+"&q="+search_string+"&l="+search_location+"&sort=&radius=&st=&jt=&start="+str(1+i*25)+"&limit=1000&fromage=&filter=&latlong=1&co=us&chnl=&userip=1.2.3.4&useragent=Mozilla/%2F4.0%28Firefox%29&v=2"

    result = urllib.urlopen(req).read()
    #result = xml.dom.minidom.parse(open(r"C:\Users\shaik\PycharmProjects\indeed\apisearch.xml"))
    result = xml.dom.minidom.parseString(result)

    def getNodeName (data) :
        nodes = []
        for nd in data[0].childNodes :
            if nd.nodeType == nd.ELEMENT_NODE :
                nodes.append(nd.nodeName)
        return nodes


    total_res = result.getElementsByTagName("result")
    nodes = getNodeName(total_res)

    final_result = {}
    for count in range(len(total_res)) :
        res_count = "result_"+str(count+i*25)
        final_result[res_count] = {}
        for each_node in nodes :
            try :
                final_result[res_count][each_node] = getText(total_res[count].getElementsByTagName(each_node)[0].childNodes)
            except :
                final_result[res_count] = {}
                pass
            
    

    out_final.update(final_result)




In [160]:
import pandas as pd

In [161]:
df = pd.DataFrame(out_final)

In [162]:
lat = list(df.loc['latitude'])
lon = list(df.loc['longitude']) 
lat = [float(i) for i in lat]
lon = [float(i) for i in lon]

In [163]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt

In [164]:
map = Basemap(projection="mill",llcrnrlat = -60,urcrnrlat=90,llcrnrlon=-180,urcrnrlon=180,resolution="c")

In [165]:
map.drawcoastlines()
map.drawcountries()
map.fillcontinents(color="#04BAE3")
map.drawmapboundary()
#map.bluemarble()

In [166]:
x,y = map(lon,lat)

In [167]:
map.plot(x,y,'ro')

In [168]:
plt.show()